# Client_sample 

Při vytvaření clienta na auth serveru jsme ziskali dulezite kody. ClientID a ClientSecret. Oba je potreba na pevno priřadit k promennym. Dale se s nimi pracuje. 

Z modulu importujeme funkce pro asynchroni request komunikaci

In [ ]:
import tornado.ioloop
import tornado.web
import json
import aiohttp
from moduls import * # Contains a module for asynchronous requests

g_ClientID="MrVDUUQ3rdOthdg0T8tvNm68"
g_ClientSecret="T7pKWd3hPYBnz7GLj7akMItPJq91NbGTTnIevUv269LaF0So"


V promenne g_port definujeme na jakem portu nam client pobezi.
G_authServerPublic je promenna typu string ve ktere mame zapsane host + port authorizacniho serveru na ktery se prepojujeme pomoci redirect
G_authServerDocker je promenna typu string ve ktere mame zapsane host + port authorizacniho serveru pro fukce get/post v pripade ze auth server bezi v dockeru. Pokud v dockeru nebezi je nutne promennou nahradit hodnotou g_authServerPublic.

In [ ]:
g_port = 9999
g_authServerPublic = "http://127.0.0.1:5000"
g_authServerDocker = "http://172.16.238.1:5000" # Adress for docker inner tunneling

'''**************************************************'''



## Uvodni strana

Primitivni uvodni strana, kde se nachazi dve tlacitka. Funkcni je pouze tlacitko na prepojeni na authserver. Pri redirect posilame na auth server informace o tom co pozadujeme, kdo jsme a jakou metodou to pozadujeme. response_type=code -> Budeme se bavit tokem authorization code. To znamena ze dostaneme athorizacni kod, ktery pam smenime za access token. Scope=profile -> Celkove pozadujeme informace o uzivateli typu profile. Vice definovano v authserver

In [ ]:
class loginPage(tornado.web.RequestHandler):
	def get(self):
		self.render("templates/loginPage.html")

	def post(self):
		'''Redirect user to authorization server, you will receive a code (request token) in redirect URL'''
		if self.get_argument("authSer_button", None) != None:
			self.redirect(tornado.httputil.url_concat(g_authServerPublic + "/oauth/authorize", 
			[("response_type","code"), 
			("client_id",g_ClientID),
			("scope","profile")]))
			
		if self.get_argument("googleAuth_button", None) != None:
			self.write("This part has not been programmed yet.")
		



# AfterLog

Sem jsme prepojeni po uspesne authorizaci uzivatele na authorizacnim serveru. Pri prepojeni ziskavame auth code, ktery nasledne menime za access token. Opet pridavame informace o tom kdo jsme, a co chceme. Aby se nestalo ze nekdo jiny nahodne vygeneruje nas auth code a pouzije ho misto nas.

Kdyz ziskame access token tak ten uz konecne muzeme vyuzit k ziskani dat o uzivateli. Jednoduse metodou get, kde jen pripojime jako parametr ziskany access token.

Po ziskani dat se prepojime na nasi hlavni stranu s prislusnymi parametry.

In [ ]:
class afterLog(tornado.web.RequestHandler):
	async def get(self):
		'''Exchange code (request token) for access token, fetch user data'''
		code = self.get_query_argument("code")
		print("Acquired code: " + str(code))

		# Token exchange
		files = {
			'grant_type': 'authorization_code',
			'scope': 'profile',
			'code': code,
		}
		response = await post_request_with_files(g_authServerDocker + "/oauth/token", data=files, ClientID=g_ClientID, ClientSecret=g_ClientSecret)
		access_token = response["access_token"]
		print("Acquired access token: " + access_token)
		
		# Fetch user data
		headers = {
			'Authorization': f"Bearer {access_token}",
		}
		response = await get_request_with_headers(g_authServerDocker + "/api/me", data=headers)
		print("Data o uzivateli ziskana: " + str(response))

		data = {"username":response["username"],  "access_token":access_token}
		self.redirect(tornado.httputil.url_concat('mainPage', data))



# Hlavni strana

Tlacitkem logout odesilame na auth server pozadavek o zruseni access tokenu. Jako parametry udavame nase ID a samozrejme access token, ktery chceme nechat smazat

In [ ]:
class mainPage(tornado.web.RequestHandler):
	def get(self):
		self.render("templates/afterLog.html", title="Client Main Page", data=self)

	async def post(self):
		'''Log out and send a request to delete access token'''
		if self.get_argument("logOut_button", None) != None:
			print("logOut")

			data = {
				'access_token': self.get_argument("access_token"),
				'client_ID':g_ClientID,
			}
			await get_request_with_params(g_authServerDocker + '/delete_AT', data)

			self.redirect('/')

		if self.get_argument("nothing_button", None) != None:
			self.write("What did you expect? Told you, I do nothing.")





prepojovani do prislusnych trid na zakladade url stromu

In [ ]:
application = tornado.web.Application([
	(r"/", loginPage),
	(r"/afterLog", afterLog),
	(r"/mainPage", mainPage),
])

hlavni kod aplikace. Spousti server a definuje port na kterem bude poslouchat.

In [ ]:
if __name__ == "__main__":
	application.listen(g_port)
	print("Client_sample running on port: " + str(g_port) + "...")
	tornado.ioloop.IOLoop.instance().start()

In [ ]:


async def get_request(url):
	async with aiohttp.ClientSession() as session:
		async with session.get(url) as resp:
			return await resp.text()
async def get_request_with_headers(url, data):
    async with aiohttp.ClientSession() as session:
        async with session.get(url=url, headers=data) as resp:
            return await resp.json()
async def post_request_with_files(url, data, ClientID, ClientSecret):
	async with aiohttp.ClientSession(auth=aiohttp.BasicAuth(ClientID,ClientSecret)) as session:
		async with session.post(url, data=data) as resp:
			return await resp.json()
async def get_request_with_params(url, data):
    async with aiohttp.ClientSession() as session:
        async with session.get(url=url, params=data) as resp:
            return await resp.text()	
